In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from scipy.optimize import leastsq
from numpy import exp
from scipy import ndimage
from func import *
from astropy.table import Table, hstack
import os
import glob
from astropy.table import Table, vstack, join
import pandas as pd
from shutil import copyfile

In [2]:
def scale_factor_spectrum(z,data,error,wave):
    k = 1+z
    select1 = (wave>6750*k) & (wave<6800*k)
    cont_dat = data[select1]
    cont_err = error[select1]
    error_maincube = np.std(cont_dat)
    error_errcube = np.mean(cont_err)
    fact = error_maincube/error_errcube
    return fact

In [3]:
def aperture_spec_fixkin_fit(int_spectrum,int_error,mini_wave,fixed_param,broad2=False,MC_loops=0):
    if broad2:
        full_gauss_fixkin = full_gauss2_fixkin
    else:
        full_gauss_fixkin = full_gauss1_fixkin
    (spectrum,error) = (int_spectrum,int_error)   
    popt_full_fit,pcov_full_fit = leastsq(full_gauss_fixkin,x0=[0.1,0.1,5,5,0.01,0.01,0.01,0.01,-0.7,0.001],args=(mini_wave,spectrum,error,fixed_param),maxfev = 10000000)       
    fitted=(full_gauss_fixkin(popt_full_fit,mini_wave,spectrum,error,fixed_param))*(error)+spectrum 
    residual = spectrum - fitted
    
    spec_parameters_MC = np.zeros((len(popt_full_fit),MC_loops))
    for l in range(MC_loops):
        iteration_data = np.random.normal(spectrum,error)   
        popt_spec_MC,pcov_spec_MC = leastsq(full_gauss_fixkin,x0=[0.1,0.1,5,5,0.01,0.01,0.01,0.01,-0.7,0.001],args=(mini_wave,iteration_data,error,fixed_param),maxfev = 10000000)
        spec_parameters_MC[:,l]=popt_spec_MC
    spec_parameters_err = np.std(spec_parameters_MC,1)
    return popt_full_fit,spec_parameters_err,fitted,residual

def central_table(obj,output_par,output_par_err,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    column_names={'amp_Hb':0,'amp_OIII5007':1,'amp_OIII5007_br':2,'amp_Hb_br':3,'amp_Hb1':4,'amp_Fe5018_1':5,'amp_Hb2':6,'amp_Fe5018_2':7,'m':8,'c':9}
    columns=[]
    for key in column_names.keys():
        columns.append(fits.Column(name=key,format='E',array=[output_par[column_names[key]]]))
        columns.append(fits.Column(name=key+'_err',format='E',array=[output_par_err[column_names[key]]]))
    coldefs = fits.ColDefs(columns)
    hdu = fits.BinTableHDU.from_columns(coldefs)
    hdu.writeto('%s/%s/%s_spectro_central_fit.fits'%(destination_path_cube,obj,obj),overwrite=True)

def central_table_with_source(obj,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    t1 = Table.read('%s/%s/source_%s.fits'%(destination_path_cube,obj,obj),format='fits')
    t2 = Table.read('%s/%s/%s_spectro_central_fit.fits'%(destination_path_cube,obj,obj),format='fits')
    new = hstack([t1, t2])
    new.write('%s/%s/%s_fluxdensity3.fits'%(destination_path_cube,obj,obj),overwrite=True)
    


In [6]:
def spectroastrometric_script(obj,z,broad2,MC_loops=100,destination_path_cube="/home/mainak/Downloads/Outflow_paper1/MUSE"):
    k = 1+z
    print ('%s'%(obj))
    fixed_param = fixed_parameters(obj)
    (mini_cube,mini_err,mini_wave,mini_header) = loadCube('%s/%s/%s.3_arcsec_minicube.fits'%(destination_path_cube,obj,obj))
    (int_spectrum,int_error) = int_spec(mini_cube,mini_err)
    (output_par,output_par_err,fitted,residual) = aperture_spec_fixkin_fit(int_spectrum,int_error,mini_wave,fixed_param,broad2,MC_loops=100)
    print (output_par,output_par_err)
    central_table(obj,output_par,output_par_err)
    central_table_with_source(obj)

In [7]:
z = {"HE0021-1810":0.05352,"HE0021-1819":0.053197,"HE0040-1105":0.041692,"HE0108-4743":0.02392,"HE0114-0015":0.04560
    ,"HE0119-0118":0.054341,"HE0212-0059":0.026385,"HE0224-2834":0.059800,"HE0227-0913":0.016451,"HE0232-0900":0.043143
    ,"HE0253-1641":0.031588,"HE0345+0056":0.031,"HE0351+0240":0.036,"HE0412-0803":0.038160,"HE0429-0247":0.042009
    ,"HE0433-1028":0.035550,"HE0853+0102":0.052,"HE0934+0119":0.050338,"HE1011-0403":0.058314,"HE1017-0305":0.049986
    ,"HE1029-1831":0.040261,"HE1107-0813":0.058,"HE1108-2813":0.024013,"HE1126-0407":0.061960,"HE1237-0504":0.009
    ,"HE1248-1356":0.01465,"HE1330-1013":0.022145,"HE1353-1917":0.035021,"HE1417-0909":0.044,"HE2128-0221":0.05248
    ,"HE2211-3903":0.039714,"HE2222-0026":0.059114,"HE2233+0124":0.056482,"HE2302-0857":0.046860}

objs = z.keys()

broad2= {'HE0021-1810':False,'HE0021-1819':False,'HE0040-1105':False,'HE0108-4743':True,'HE0114-0015':False
        ,'HE0119-0118':True,'HE0212-0059':True,'HE0224-2834':False,'HE0227-0913':True,'HE0232-0900':False
        ,'HE0253-1641':True,'HE0345+0056':True,'HE0351+0240':True,'HE0412-0803':False,'HE0429-0247':True
        ,'HE0433-1028':True,'HE0853+0102':True,'HE0934+0119':True,'HE1011-0403':True,'HE1017-0305':False
        ,'HE1029-1831':True,'HE1107-0813':True,'HE1108-2813':False,'HE1126-0407':True,'HE1237-0504':False
        ,'HE1248-1356':False,'HE1330-1013':True,'HE1353-1917':True,'HE1417-0909':False,'HE2128-0221':False
        ,'HE2211-3903':False,'HE2222-0026':True,'HE2233+0124':True,'HE2302-0857':True}


for obj in objs:
    spectroastrometric_script(obj,z[obj],broad2[obj])   

HE0021-1810
[ 1.15063547e+00  1.68428693e+01  1.06279909e+01  4.23057118e-01
  1.31720716e+00  1.00000000e-02  1.98749214e+00  1.00000000e-02
  1.84484097e+00 -1.83767316e+00] [1.32860773e-01 2.27183608e-01 1.32076363e-01 9.06424182e-02
 7.84572485e-02 1.73472348e-18 4.70357328e-02 1.73472348e-18
 2.82184066e-01 1.50038856e+00]
HE0021-1819
[ 4.42914320e+00  4.52739499e+01  8.41710849e+00  1.19673914e+00
  1.35764044e+00  1.00000000e-02  1.92273473e-01  1.00000000e-02
 -9.04256400e-01  6.23888175e+00] [3.33046610e-02 5.68623602e-02 1.84418011e-02 1.98859347e-02
 8.47200197e-03 1.73472348e-18 5.34464601e-03 1.73472348e-18
 1.98451149e-02 1.02656768e-01]
HE0040-1105
[ 8.63055790e+00  8.97375856e+01  1.33096680e+01  1.90862658e+00
  7.57423291e+00  1.00000000e-02  9.52971652e-01  1.00000000e-02
  8.98649910e-01 -1.94537634e-01] [6.00171815e-02 8.76442822e-02 4.22169149e-02 4.17835359e-02
 1.27022895e-02 1.73472348e-18 9.04816347e-03 1.73472348e-18
 2.74062045e-02 1.39195458e-01]
HE0108-474

[ 5.29609944e+00  4.00955017e+01  9.65682655e+00  2.89511768e+00
  5.44376429e+00  1.00000000e-02  6.56081862e-01  1.00000000e-02
 -4.29068469e-01  6.13521838e+00] [8.01565065e-02 9.88711731e-02 6.21294054e-02 6.45946188e-02
 1.36717960e-02 1.73472348e-18 7.74791844e-03 1.73472348e-18
 2.31584314e-02 1.19688925e-01]
HE2211-3903
[8.16540723e+00 4.61479927e+01 1.85512946e+00 2.98246009e-01
 4.52018405e+00 1.00000000e-02 7.12944545e-01 1.00000000e-02
 1.70357097e-01 1.01009846e+01] [3.91581338e-02 4.57794864e-02 2.80046741e-02 3.15548839e-02
 9.65822960e-03 1.73472348e-18 8.66160954e-03 1.73472348e-18
 3.46225310e-02 1.76151141e-01]
HE2222-0026
[ 2.24772761  8.92041825  1.6499183  -0.18471139  2.6405993   3.6364878
  0.25721126  0.50180858 -0.02250962  3.45967332] [0.03894934 0.03486606 0.02146758 0.02758883 0.01768139 0.01503402
 0.01208885 0.00992841 0.02691232 0.13976919]
HE2233+0124
[ 1.01838399e+00  8.30618521e+00  1.14303356e+00  3.77106270e-01
  1.21236184e+00  1.82253065e+00 -1.41